In [1]:
!pip install numpy pandas scikit-learn tensorflow imblearn openpyxl

In [25]:
#Importation des librairies communes (peu prendre du temps)
import os
import sys
import numpy
import pandas as pd
#Chemin menant aux données
data_path = os.path.join(os.getcwd(), "source")
#création du dictionnaire
data_dict = dict()
# Boucle pour récupérer tous les fichiers CSV dans le dossier et les ajouter au dictionnaire
for file in os.listdir(data_path):
    if file.endswith(".csv"):
        file_path = os.path.join(data_path, file)
        dict_key = file.split('.')[0]  # Nom du fichier sans l'extension .csv
        data_dict[dict_key] = pd.read_csv(file_path)
data_dict.keys()

dict_keys(['appearances', 'club_games', 'clubs', 'competitions', 'game_events', 'game_lineups', 'games', 'player_valuations', 'players', 'clubs_link_api'])

In [26]:
#Toutes les fonctions
def games_of_the_season(df_games, club_id, saison, lieu='tous'):
    """
    Récupère tous les matchs d'une saison pour un club spécifié, ou les 10 derniers matchs si moins de 5 matchs dans la saison.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    saison (str) : Saison à filtrer.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    DataFrame : Les matchs demandés avec les informations détaillées.
    """
    # Filtrage initial par saison et lieu
    if lieu == 'domicile':
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) & (df_games['season'] == saison)]
    elif lieu == 'exterieur':
        matchs_du_club = df_games[(df_games['away_club_id'] == club_id) & (df_games['season'] == saison)]
    else:
        matchs_du_club = df_games[((df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)) & (df_games['season'] == saison)]
    
    # Vérification du nombre de matchs
    if len(matchs_du_club) < 5:
        # Prendre les 10 derniers matchs toutes saisons confondues si moins de 5 matchs dans la saison
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(10)
    else:
        # Trier par date
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False)

    # Ajout des informations supplémentaires
    matchs_du_club['adversaire_id'] = matchs_du_club.apply(lambda row: row['away_club_id'] if row['home_club_id'] == club_id else row['home_club_id'], axis=1)
    matchs_du_club['lieu'] = matchs_du_club.apply(lambda row: 'domicile' if row['home_club_id'] == club_id else 'exterieur', axis=1)
    matchs_du_club['résultat'] = matchs_du_club.apply(lambda row: 'victoire' if (row['home_club_id'] == club_id and row['home_club_goals'] > row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] > row['home_club_goals']) else 'défaite' if (row['home_club_id'] == club_id and row['home_club_goals'] < row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] < row['home_club_goals']) else 'nul', axis=1)

    return matchs_du_club[['date', 'season', 'home_club_id', 'away_club_id', 'adversaire_id', 'lieu', 'home_club_goals', 'away_club_goals', 'résultat']]

def last_five_games(df_games, club_id, date, lieu='tous'):
    """
    Récupère les 5 derniers matchs d'un club spécifié avant une date donnée en fonction du lieu du match.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    date (str ou pd.Timestamp) : Date avant laquelle les matchs doivent être récupérés.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    DataFrame : Les matchs demandés avec les informations détaillées.
    """
    # Assurer que la colonne 'date' est au format datetime
    df_games['date'] = pd.to_datetime(df_games['date'])

    # Filtrage des matchs avant la date donnée
    matchs_du_club = df_games[df_games['date'] < date]
    
    if lieu == 'domicile':
        matchs_du_club = matchs_du_club[matchs_du_club['home_club_id'] == club_id]
    elif lieu == 'exterieur':
        matchs_du_club = matchs_du_club[matchs_du_club['away_club_id'] == club_id]
    else:  # 'tous'
        matchs_du_club = matchs_du_club[(matchs_du_club['home_club_id'] == club_id) | (matchs_du_club['away_club_id'] == club_id)]
    
    # Tri des matchs par date et sélection des 5 derniers
    matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(5)

    # Ajout des informations supplémentaires
    matchs_du_club['adversaire_id'] = matchs_du_club.apply(lambda row: row['away_club_id'] if row['home_club_id'] == club_id else row['home_club_id'], axis=1)
    matchs_du_club['lieu'] = matchs_du_club.apply(lambda row: 'domicile' if row['home_club_id'] == club_id else 'exterieur', axis=1)
    matchs_du_club['résultat'] = matchs_du_club.apply(lambda row: 'victoire' if (row['home_club_id'] == club_id and row['home_club_goals'] > row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] > row['home_club_goals']) else 'défaite' if (row['home_club_id'] == club_id and row['home_club_goals'] < row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] < row['home_club_goals']) else 'nul', axis=1)

    return matchs_du_club[['date', 'home_club_id', 'away_club_id', 'adversaire_id', 'lieu', 'home_club_goals', 'away_club_goals', 'résultat']]
    
def last_game_id(df_games, club_id):
    """
    Récupère l'ID du dernier match joué par un club spécifié.

    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.

    Retourne:
    int : ID du dernier match joué par le club.
    """
    # Filtrer les matchs impliquant le club
    matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
    
    # Trier par date en ordre décroissant et prendre le premier match
    dernier_match = matchs_du_club.sort_values(by='date', ascending=False).iloc[0]

    return dernier_match['game_id']

def match_sheet(df_game_lineups, match_id, club_id):
    """
    Récupère tous les player_id qui ont participé à un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs qui ont participé au match.
    """
    # Filtrer pour obtenir les lignes correspondant au match_id et club_id spécifiés
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & (df_game_lineups['club_id'] == club_id)]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

def starting_lineup_match(df_game_lineups, match_id, club_id):
    """
    Récupère les player_id des joueurs qui étaient dans la composition de départ d'un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs de la composition de départ.
    """
    #df_game_lineups = data_dict['game_lineups']
    # Filtrer pour obtenir les lignes correspondant au match_id, club_id et type 'starting_lineup'
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & 
                                      (df_game_lineups['club_id'] == club_id) & 
                                      (df_game_lineups['type'] == 'starting_lineup')]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

def market_value_player(df_player, player_id):
    """
    Récupère la valeur marchande d'un joueur spécifié.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    player_id (int) : ID du joueur.
    
    Retourne:
    float : La valeur marchande du joueur en euros.
    """
    # Trouver l'entrée correspondant au player_id
    valeur_joueur = df_player[df_player['player_id'] == player_id]['market_value_in_eur']

    # Retourner la valeur marchande, ou None si le joueur n'est pas trouvé
    return valeur_joueur.iloc[0] if not valeur_joueur.empty else None

def mean_team_value(df_player, liste_player_id):
    """
    Calcule la valeur marchande moyenne d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Valeur marchande moyenne de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs) / len(valeurs) if valeurs else 0

def sum_team_value(df_player, liste_player_id):
    """
    Calcule la somme des valeurs marchandes d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Somme des valeurs marchandes de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs)

def predire_resultat(model, match_features, scaler, seuil_nul=0.5):
    """
    Prédit le résultat d'un match de football en utilisant un modèle de réseau de neurones entraîné.
    
    Paramètres :
    model : Le modèle de réseau de neurones entraîné.
    match_features : Un DataFrame contenant les caractéristiques du match.
    scaler : L'objet StandardScaler utilisé pour normaliser les caractéristiques d'entraînement.
    seuil_nul : Seuil de confiance pour prédire un match nul.
    
    Retourne :
    str : La prédiction du résultat du match ('victoire', 'nul', 'défaite').
    """
    # Normalisation des caractéristiques du match
    match_features_scaled = scaler.transform(match_features)

    # Faire la prédiction
    prediction_prob = model.predict(match_features_scaled)
    return prediction_prob[0]
    """
    # Obtenir l'indice de la classe avec la probabilité la plus élevée
    predicted_class = np.argmax(prediction_prob, axis=1)[0]
    

    # Ajuster la prédiction en fonction du seuil pour les matchs nuls
    if prediction_prob[0][1] >= seuil_nul:  # Supposons que l'indice 1 représente les matchs nuls
        resultat = 'nul'
    else:
        resultat = 'victoire' if predicted_class == 2 else 'défaite'  # Supposons que l'indice 2 représente les victoires

    return resultat
    """

def get_name(club_id):
    """
    Renvoie le nom du club de football basé sur son identifiant.

    Paramètres:
    club_id (int): L'identifiant du club.

    Retourne:
    str: Le nom du club.
    """
    # Trouver la ligne correspondante au club_id
    club_info = df_clubs[df_clubs['club_id'] == club_id]

    # Renvoie le nom du club s'il est trouvé, sinon renvoie None
    if not club_info.empty:
        return club_info['name'].iloc[0]
    else:
        return None

def calculer_forme(df_games, club_id, date_du_match, lieu='tous'):
    """
    Calcule la forme d'un club sur ses 5 derniers matchs avant la date du match spécifiée.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    date_du_match (str ou pd.Timestamp) : Date du match pour lequel la forme est calculée.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    int : Les points accumulés par le club sur ses 5 derniers matchs.
    """
    # Assurer que la colonne 'date' est au format datetime
    #df_games['date'] = pd.to_datetime(df_games['date'])

    # Filtrer les matchs précédents et trier par date
    matchs_precedents = last_five_games(df_games, club_id, date_du_match, lieu)
    
    # Calculer les points (exemple simplifié)
    points = 0
    for _, row in matchs_precedents.iterrows():
        if row['home_club_id'] == club_id:
            points += 3 if row['home_club_goals'] > row['away_club_goals'] else 1 if row['home_club_goals'] == row['away_club_goals'] else 0
        else:
            points += 3 if row['away_club_goals'] > row['home_club_goals'] else 1 if row['away_club_goals'] == row['home_club_goals'] else 0
    
    return points

In [27]:
import pandas as pd

df_final_path = "df_final.xlsx"
df_to_train_path = "df_to_train.xlsx"

# Charger le fichier Excel dans un DataFrame
df_final = pd.read_excel(df_final_path, engine='openpyxl')
df_to_train = pd.read_excel(df_to_train_path, engine='openpyxl')
conditions = (
    (df_final['home_team_market_value'] >= 1) & 
    (df_final['away_team_market_value'] >= 1) & 
    (df_final['home_team_form'] >= 1) & 
    (df_final['away_team_form'] >= 1)
)
df_to_train = df_final[conditions]
df_to_train = df_to_train.drop(["season", "game_id", "away_club_position", "home_club_position", "date"], axis=1).dropna()

df_to_train.head()

FileNotFoundError: [Errno 2] No such file or directory: 'df_final.xlsx'

In [5]:
print(f"Taille du df {len(df_to_train)}")
print("Nombre de lignes avec 0 dans 'resultat':", (df_to_train['resultat'] == 0).sum())
print("Nombre de lignes avec -1 dans 'resultat':", (df_to_train['resultat'] == -1).sum())
print("Nombre de lignes avec 1 dans 'resultat':", (df_to_train['resultat'] == 1).sum())
print("PB home market value:", (df_to_train['home_team_market_value'] < 1).sum())
print("PB away market value:", (df_to_train['away_team_market_value'] < 1).sum())
print("PB home market value:", (df_to_train['home_team_form'] < 1).sum())
print("PB away market value:",  (df_to_train['away_team_form'] < 1).sum())

Taille du df 33640
Nombre de lignes avec 0 dans 'resultat': 8349
Nombre de lignes avec -1 dans 'resultat': 10326
Nombre de lignes avec 1 dans 'resultat': 14965
PB home market value: 0
PB away market value: 0
PB home market value: 0
PB away market value: 0


In [6]:
#Configurations et imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df_clubs = data_dict["clubs"]
df_players = data_dict['players']
df_games = data_dict['games']
df_game_lineups = data_dict['game_lineups']

In [7]:
df_final = df_final[df_final["season"] == 2022]
df_final.head()

,game_id,season,date,home_club_id,away_club_id,home_club_position,away_club_position,resultat,home_team_market_value,away_team_market_value,home_team_form,away_team_form
40102,3842706,2022,2022-07-15,3719,964,11,9,0,3850000.0,81400000.0,11,7
40103,3828412,2022,2022-07-15,865,5724,7,8,0,23700000.0,8100000.0,6,5
40104,3842704,2022,2022-07-16,3725,232,12,7,0,14700000.0,47000000.0,6,6
40105,3842705,2022,2022-07-16,14589,2696,14,1,-1,9050000.0,30250000.0,4,3
40106,3842700,2022,2022-07-16,2410,11127,3,16,1,61000000.0,11400000.0,7,7


In [8]:
df_games['date'] = pd.to_datetime(df_games['date'])

In [19]:
# Création des caractéristiques pour chaque match
features = []
labels = []

for _, row in df_final.iterrows():
    home_forme = calculer_forme(df_games, row['home_club_id'], row["date"], "domicile")
    away_forme = calculer_forme(df_games, row['away_club_id'], row["date"], "exterieur")
    home_valeur_marchande = sum_team_value(df_players, starting_lineup_match(df_game_lineups, row["game_id"], row['home_club_id']))
    away_valeur_marchande = sum_team_value(df_players, starting_lineup_match(df_game_lineups, row["game_id"], row['away_club_id'])) 
    
    features.append([home_forme, away_forme, home_valeur_marchande, away_valeur_marchande])
    
    labels.append(row["resultat"])

# Convertir en DataFrame pour l'utilisation avec des modèles de machine learning
df_features = pd.DataFrame(features, columns=['home_forme', 'away_forme', 'home_valeur_marchande', 'away_valeur_marchande'])
df_labels = pd.Series(labels)
print(df_labels)
print(df_features)

0       0
1       0
2       0
3      -1
4       1
       ..
4067    0
4068    0
4069    1
4070    1
4071   -1
Length: 4072, dtype: int64
      home_forme  away_forme  home_valeur_marchande  away_valeur_marchande
0             11           7              8050000.0            126100000.0
1              6           5             44900000.0              7350000.0
2              6           6             16900000.0             38700000.0
3              4           3              5800000.0             32700000.0
4              7           7             58100000.0              8650000.0
...          ...         ...                    ...                    ...
4067           7           4              7400000.0             14025000.0
4068          15          10             53800000.0             14500000.0
4069           7           4              6075000.0              3525000.0
4070           7           6              8700000.0             20875000.0
4071           8           9          

In [20]:
# Supprimer les lignes contenant des valeurs NaN
df_features_clean = df_features.dropna()
df_labels_clean = df_labels[df_features_clean.index]
df_features_clean
df_labels_clean

0       0
1       0
2       0
3      -1
4       1
       ..
4067    0
4068    0
4069    1
4070    1
4071   -1
Length: 4071, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df_features_clean, df_labels_clean, test_size=0.1, random_state=42)

# Création et entraînement du modèle
model = LogisticRegression()
model.fit(X_train, y_train)

# Évaluation du modèle
predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 0.5514705882352942


In [22]:
def calculer_probabilites_victoire(df_games, df_players, model, liste_matchs):
    """
    Calcule les probabilités de victoire pour une liste de matchs.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    df_players (DataFrame) : DataFrame des joueurs et de leurs valeurs marchandes.
    model (Model) : Modèle de machine learning entraîné.
    liste_matchs (List[Tuple[int, int]]) : Liste de tuples (home_club_id, away_club_id).
    
    Retourne:
    DataFrame : Probabilités de victoire pour chaque match.
    """
    # Liste pour stocker les caractéristiques de chaque match
    features_list = []
    
    # Préparer les caractéristiques pour chaque match
    for home_club_id, away_club_id in liste_matchs:
        home_forme = calculer_forme(df_games, home_club_id, "2024-01-01", "domicile")
        away_forme = calculer_forme(df_games, away_club_id, "2024-01-01", "exterieur")
        home_valeur_marchande = sum_team_value(df_players, starting_lineup_match(df_game_lineups, last_game_id(df_games, home_club_id), home_club_id))
        away_valeur_marchande = sum_team_value(df_players, starting_lineup_match(df_game_lineups, last_game_id(df_games, away_club_id), away_club_id))
        
        # Ajouter les caractéristiques pour le match courant à la liste
        features_list.append([home_club_id, away_club_id, home_forme, away_forme, home_valeur_marchande, away_valeur_marchande])
    
    # Créer un DataFrame avec toutes les caractéristiques
    df_features = pd.DataFrame(features_list, columns=['home_club_id', 'away_club_id', 'home_forme', 'away_forme', 'home_valeur_marchande', 'away_valeur_marchande'])
    
   # Préparer les caractéristiques pour la prédiction
    match_features = df_features[['home_forme', 'away_forme', 'home_valeur_marchande', 'away_valeur_marchande']]
    
    # Prédiction des probabilités avec le modèle pour chaque match
    probas = model.predict_proba(match_features)
    
    # Ajouter toutes les probabilités prédites au DataFrame
    for i, class_label in enumerate(model.classes_):
        df_features[f'prob_{class_label}'] = probas[:, i]
    
    # Sélectionner uniquement les colonnes 'home_club_id', 'away_club_id', et celles qui commencent par 'prob_'
    df_probabilites = df_features.filter(regex='^(home_club_id|away_club_id|prob_|home_forme|away_forme)')
    
    return df_probabilites

# Liste des matchs sous forme de tuples (home_club_id, away_club_id)
liste_matchs = [(1041, 583), (1041, 3524), (3524, 1041), (583, 3524), (1041, 162), (162, 1041), (418, 3302), (418, 12321), (31, 281), (281, 31), (418, 281), (281, 418)]  # Ajoutez d'autres matchs comme nécessaire

# Appel de la fonction et impression des résultats
df_players = data_dict["players"]
df_probabilites = calculer_probabilites_victoire(df_games, df_players, model, liste_matchs)
df_probabilites

,home_club_id,away_club_id,home_forme,away_forme,prob_-1,prob_0,prob_1
0,1041,583,7,8,0.879980,0.088631,0.031389
1,1041,3524,7,5,0.217513,0.295244,0.487244
2,3524,1041,4,6,0.418653,0.306162,0.275185
3,583,3524,13,5,0.002023,0.026219,0.971759
4,1041,162,7,7,0.384939,0.285520,0.329540
5,162,1041,12,6,0.198498,0.263349,0.538153
6,418,3302,15,0,0.000821,0.014999,0.984180
7,418,12321,15,13,0.001723,0.019684,0.978594
8,31,281,11,12,0.568973,0.142940,0.288088
9,281,31,11,10,0.009182,0.033882,0.956936


In [13]:
"""
1041 = OL
583 = PSG
3524 = Clermont
162 = Monaco
418 = Real Madrid
3302 = Almeria
12321 = Girona
31 = Liverpool
281 = ManchesterCity
"""

'\n1041 = OL\n583 = PSG\n3524 = Clermont\n162 = Monaco\n418 = Real Madrid\n3302 = Almeria\n12321 = Girona\n31 = Liverpool\n281 = ManchesterCity\n'

In [23]:
def predire_resultat(model, match_features):
    # Faire la prédiction
    prediction_prob = model.predict_proba(match_features)
    return prediction_prob[0]

In [24]:
df_clubs = data_dict["clubs"]
df_players = data_dict['players']
df_games = data_dict['games']
df_game_lineups = data_dict['game_lineups']

# Liste des matchs pour lesquels faire des prédictions
#liste_matchs = [(1041, 583), (1041, 3524), (583, 3524), (1041, 162), (162, 1041), 
#                (418, 3302), (418, 12321), (31, 281), (281, 31), (418, 281), (281, 418), (281, 281)]
import random

# Liste des identifiants de club
club_ids = [
    162, 244, 273, 347, 415, 417, 583, 667, 738, 826,
    969, 995, 1041, 1082, 1158, 1421, 3524, 3911
]

# Générer des paires aléatoires de matchs
liste_matchs = []

while len(club_ids) > 1:  # Continue tant qu'il y a au moins 2 clubs pour former un match
    home = random.choice(club_ids)
    club_ids.remove(home)
    away = random.choice(club_ids)
    club_ids.remove(away)
    liste_matchs.append((home, away))

# Boucle sur chaque match pour faire des prédictions
for home_club_id, away_club_id in liste_matchs:
    match_features = pd.DataFrame({
        'home_forme': [calculer_forme(df_games, home_club_id, "2024-01-01", "domicile")],
        'away_forme': [calculer_forme(df_games, away_club_id, "2024-01-01", "exterieur")],
        'home_valeur_marchande': [sum_team_value(df_players, starting_lineup_match(df_game_lineups, last_game_id(df_games, home_club_id), home_club_id))],
        'away_valeur_marchande': [sum_team_value(df_players, starting_lineup_match(df_game_lineups, last_game_id(df_games, away_club_id), away_club_id))]
    })

    # Faire la prédiction pour le match courant
    resultat_predit = predire_resultat(model, match_features)
    print(f"Résultat prédit pour le match entre les clubs {get_name(home_club_id)} et {get_name(away_club_id)} : {resultat_predit}")

Résultat prédit pour le match entre les clubs Toulouse Football Club et Montpellier Hérault Sport Club : [0.32648483 0.31192827 0.3615869 ]
Résultat prédit pour le match entre les clubs Clermont Foot 63 et Racing Club de Lens : [0.48768866 0.28632167 0.22598968]
Résultat prédit pour le match entre les clubs Paris Saint-Germain Football Club et Stade Rennais Football Club : [0.00507389 0.0365856  0.95834051]
Résultat prédit pour le match entre les clubs Le Havre Athletic Club et Racing Club de Strasbourg Alsace : [0.37434154 0.31052621 0.31513226]
Résultat prédit pour le match entre les clubs Football Club Lorient-Bretagne Sud et Association sportive de Monaco Football Club : [0.50065326 0.28013961 0.21920713]
Résultat prédit pour le match entre les clubs Olympique Gymnaste Club Nice Côte d'Azur et Lille Olympique Sporting Club Lille Métropole : [0.34687568 0.2814881  0.37163622]
Résultat prédit pour le match entre les clubs Olympique Lyonnais et Olympique de Marseille : [0.31065271 0.2

In [16]:
import joblib
# Supposons que `model` est votre modèle entraîné
joblib.dump(model, 'model_rl.pkl')

['model_rl.pkl']